In [2]:
import chromadb
import os
import json
# Uso questa libreria per cambiare LLM
from chromadb.utils import embedding_functions
chroma_client = chromadb.Client()

In [3]:
# Vado a recuperare dal file topic i topic, in modo da inserirli nel database 
with open('data/topic.json', 'r') as f:
    topic_data = json.load(f)

documents=[]
metadatas=[]
ids=[]
id=1


for topic in topic_data:
    for top in topic_data[topic]:
        metadatas.append({"source":"topic.json"})
        documents.append(top)
        ids.append(str(id))
        id+=1
    
documents

['Escursioni in montagna',
 'Viaggi in camper',
 'Turismo culturale',
 'Viaggi low-cost',
 'Destinazioni esotiche',
 'Cucina italiana',
 'Ricette vegetariane/vegane',
 'Cucina gourmet',
 'Dessert e dolci',
 'Cucina internazionale',
 'Allenamento a corpo libero',
 'Yoga e meditazione',
 'Alimentazione sana',
 'Programmi di dimagrimento',
 'Allenamento funzionale',
 'Pittura ad olio',
 'Fotografia di paesaggi',
 'Scultura',
 'Disegno e illustrazione',
 'Arte digitale',
 'Rock',
 'Hip hop',
 'Jazz',
 'Musica classica',
 'Indie',
 'Intelligenza artificiale',
 'Criptovalute',
 'App per la produttivitÃ\xa0',
 'Fotografia digitale',
 'Nuove app e software',
 'Fantascienza',
 'Drammi',
 'Commedie romantiche',
 'Thriller psicologici',
 'Anime',
 'Romanzi storici',
 'Fantasy epico',
 'Saggi di divulgazione scientifica',
 'Poesia contemporanea',
 "Libri d'arte",
 'Fotografia di viaggio',
 'Ritratti in studio',
 'Fotografia di architettura',
 'Editing di paesaggi',
 'Fotografia naturalistica',
 'P

In [5]:
# Cambio LLM usato-> di base è "all-MiniLM-L6-v2"
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
# Crea database -> gli comunico llm che voglio
collection = chroma_client.create_collection(name="my_collection", embedding_function=sentence_transformer_ef)
#collection = chroma_client.create_collection(name="my_collection")

UniqueConstraintError: Collection my_collection already exists

In [6]:
#sto anddando ad insesire 2 nuovi elementi con il rispettivo document, source ed id
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [10]:
# utilizzo la query per recuperarlo, tentendo conto che non sto andando a cercare precisamente
# quello ceh c'è scritto in un documenti, quindi verrà calcolata una sorta di distanza  
results = collection.query(
    # questa è la query che utilizzerò in base alle operazion iche devo svolgere
    query_texts=["Maturità 2024, il toto-tracce per la prima prova: gli studenti scommettono su D'Annunzio e Pirandello "],
    n_results=5,
    # Scelgo i campi che voglio mi vengano tornati
    include=['distances','metadatas','documents']
)
# la query restituirà una distance che più è piccola più il contenuto che sto cercando sarà simile a quello comaprato
results

{'ids': [['94', '77', '96', '76', '98']],
 'distances': [[0.8766587972640991,
   0.9156837463378906,
   0.9362766742706299,
   0.9479689598083496,
   0.9770452380180359]],
 'metadatas': [[{'source': 'topic.json'},
   {'source': 'topic.json'},
   {'source': 'topic.json'},
   {'source': 'topic.json'},
   {'source': 'topic.json'}]],
 'embeddings': None,
 'documents': [['Pianificazione pensionistica',
   'Cura dei gatti',
   'Consigli di relazione',
   'Addestramento dei cani',
   'Consigli per appuntamenti']],
 'uris': None,
 'data': None}